In [1]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [2]:
from pyspark.sql.functions import regexp_replace, split, map_from_arrays, lit

df=spark.range(1).withColumn("input",lit("  {   'k1'  :  'v1'  ,  'k2'  :  'v2'  }  ")).drop("id")

df.show(truncate=False)

+------------------------------------------+
input |
+------------------------------------------+
 { 'k1' : 'v1' , 'k2' : 'v2' } |
+------------------------------------------+

In [3]:
# For string in json format, uncomment & use below
# input_quote_str='"'
# For string in format similar to python dictionaries, uncomment & use below
# For the given example single quotes are used so using below value
input_quote_str="'"

In [4]:
# Remove all extra spaces around :(colon) | ,(comma) | {(open curly brace) | }(closed curly brace)
trimDF=df.selectExpr('input','regexp_replace(input,"[\\\s]*([:,{}])[\\\s]*","$1") as trimmed_input')

# Remove input quotes
if input_quote_str == "'":
  trimWithNoQuotesDF=trimDF.selectExpr('input','trimmed_input',
                                       "translate(trimmed_input,\"{}\",\"\") as trimmed_input_no_quotes".format(input_quote_str))
else:
  trimWithNoQuotesDF=trimDF.selectExpr('input','trimmed_input',
                                       'translate(trimmed_input,\'{}\',\'\') as trimmed_input_no_quotes'.format(input_quote_str))

trimWithNoQuotesDF.show(truncate=False)


+------------------------------------------+---------------------+-----------------------+
input |trimmed_input |trimmed_input_no_quotes|
+------------------------------------------+---------------------+-----------------------+
 { 'k1' : 'v1' , 'k2' : 'v2' } |{'k1':'v1','k2':'v2'}|{k1:v1,k2:v2} |
+------------------------------------------+---------------------+-----------------------+

In [5]:
mapKeyValArrayDF=trimWithNoQuotesDF.selectExpr('input','trimmed_input','trimmed_input_no_quotes',
                              "split(regexp_replace(translate(trimmed_input_no_quotes,'}{',','),':[^,]*|,$',''),',') as map_key_array_list",
                             "split(regexp_replace(regexp_replace(translate(trimmed_input_no_quotes,'{}',','),',[^:]*',''),'^:',''),':') as value_key_array_list")

mapKeyValArrayDF.show(truncate=False)

+------------------------------------------+---------------------+-----------------------+------------------+--------------------+
input |trimmed_input |trimmed_input_no_quotes|map_key_array_list|value_key_array_list|
+------------------------------------------+---------------------+-----------------------+------------------+--------------------+
 { 'k1' : 'v1' , 'k2' : 'v2' } |{'k1':'v1','k2':'v2'}|{k1:v1,k2:v2} |[k1, k2] |[v1, v2] |
+------------------------------------------+---------------------+-----------------------+------------------+--------------------+

In [6]:
finalDFwithMap=mapKeyValArrayDF.selectExpr('input','trimmed_input','trimmed_input_no_quotes'
                                           ,'map_key_array_list','value_key_array_list',
                                           "map_from_arrays(map_key_array_list,value_key_array_list) as keyValMap")

finalDFwithMap.show(truncate=False)

+------------------------------------------+---------------------+-----------------------+------------------+--------------------+--------------------+
input |trimmed_input |trimmed_input_no_quotes|map_key_array_list|value_key_array_list|keyValMap |
+------------------------------------------+---------------------+-----------------------+------------------+--------------------+--------------------+
 { 'k1' : 'v1' , 'k2' : 'v2' } |{'k1':'v1','k2':'v2'}|{k1:v1,k2:v2} |[k1, k2] |[v1, v2] |[k1 -> v1, k2 -> v2]|
+------------------------------------------+---------------------+-----------------------+------------------+--------------------+--------------------+